In [ ]:
!pip install  torchxrayvision

import torchxrayvision

In [ ]:
#Code is first test on https://github.com/mlmed/torchxrayvision
import torchxrayvision as xrv
import skimage, torch, torchvision
import matplotlib.pyplot as plt
import os
import tabulate

#added code
from google.colab import drive
from PIL import Image
from os import listdir
from tabulate import tabulate
drive.mount('/content/drive')
#end of added code

path = "/content/drive/MyDrive/Shared Capstone/Colab/starting_images"

file_name = "00000003_000.png"

index = -1

with open(r"/content/drive/MyDrive/Shared Capstone/Colab/Data_Entry_2017.csv", 'r') as fp:
	lines = len(fp.readlines())
	print('Total Number of lines:', lines)

with open("/content/drive/MyDrive/Shared Capstone/Colab/Data_Entry_2017.csv") as myFile:
    for num, line in enumerate(myFile, 0):
        if file_name in line:
            index = num
            print("the line content is: ", line)

print("index = ", index)

# Prepare the image:
original_img = Image.open(path + "/" + file_name)
modified_img = original_img.convert('RGB')
modified_img.save('image.jpg')

img = skimage.io.imread('image.jpg')
plt.imshow(img);
img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
img = img.mean(2)[None, ...] # Make single color channel

transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
img = transform(img)
img = torch.from_numpy(img)

# Load model

model = xrv.models.DenseNet(weights="densenet121-res224-nih") # NIH chest X-ray8

d_nih = xrv.datasets.NIH_Dataset(imgpath=path, views=["PA","AP"], unique_patients=False)

print("\n")
print(d_nih.csv.iloc[index])

sample = d_nih[index]
plt.imshow(sample["img"][0], cmap="Greys_r");
actual_dictionary = dict(zip(d_nih.pathologies,sample["lab"]))

outputs = model(img[None,...]) # or outputs = model.features(img[None,...])

# Print results

probability_dictionary = dict(zip(model.pathologies,outputs[0].detach().numpy()))

print("\n")

array1 = ['Atelectasis', probability_dictionary['Atelectasis'], actual_dictionary['Atelectasis']]
array2 = ['Cardiomegaly', probability_dictionary['Cardiomegaly'], actual_dictionary['Cardiomegaly']]
array3 = ['Consolidation', probability_dictionary['Consolidation'], actual_dictionary['Consolidation']]
array4 = ['Edema', probability_dictionary['Edema'], actual_dictionary['Edema']]
array5 = ['Effusion', probability_dictionary['Effusion'], actual_dictionary['Effusion']]

table = [['Disease', 'Probabilty', 'Actual'], array1, array2, array3, array4, array5]
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
